<a href="https://colab.research.google.com/github/tharakats/BankLoanEvaluator/blob/main/Model/BankLoanEvaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Copyright 2020 Tharaka Sampath

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

In [2]:
import numpy as np
import pandas as pd

In [3]:
dataset = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT3dBWyWxVw2rMsVnNaG36DWb33ecoq1Jd9i99ogk7S9mKDirDdg359NvLBmi5IlqLU9XFH4GLx6n_d/pub?gid=1809176138&single=true&output=csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X=dataset.iloc[:,3:-1].values
Y=dataset.iloc[:,-1].values

In [5]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan,strategy='mean')
X[:,4:12]=imputer.fit_transform(X[:,4:12])

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=1)

In [7]:
X_train

array([[628, 'Germany', 'Male', ..., 0.0, 1.0, 124749.08],
       [626, 'France', 'Female', ..., 0.0, 0.0, 41104.82],
       [612, 'Germany', 'Female', ..., 1.0, 1.0, 45750.21],
       ...,
       [672, 'France', 'Female', ..., 1.0, 1.0, 92027.69],
       [663, 'France', 'Female', ..., 1.0, 1.0, 101168.9],
       [763, 'Germany', 'Male', ..., 1.0, 0.0, 33462.94]], dtype=object)

In [8]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,StandardScaler
gender=LabelEncoder()
gender_x=gender.fit(X_train[:,2])

X_train[:,2]=gender_x.transform(X_train[:,2])
X_test[:,2]=gender_x.transform(X_test[:,2])

country=LabelEncoder()
country_x=country.fit(X_train[:,1])
X_train[:,1]=country_x.transform(X_train[:,1])
X_test[:,1]=country_x.transform(X_test[:,1])

In [9]:
X_train

array([[628, 1, 1, ..., 0.0, 1.0, 124749.08],
       [626, 0, 0, ..., 0.0, 0.0, 41104.82],
       [612, 1, 0, ..., 1.0, 1.0, 45750.21],
       ...,
       [672, 0, 0, ..., 1.0, 1.0, 92027.69],
       [663, 0, 0, ..., 1.0, 1.0, 101168.9],
       [763, 1, 1, ..., 1.0, 0.0, 33462.94]], dtype=object)

In [10]:
country_onehot=OneHotEncoder()
country_onehot_x=country_onehot.fit(X_train[:,1].reshape(-1,1))
country_onehot_x_train=country_onehot_x.transform(X_train[:,1].reshape(-1,1)).toarray()
country_onehot_x_test=country_onehot_x.transform(X_test[:,1].reshape(-1,1)).toarray()

X_train=np.concatenate((country_onehot_x_train,X_train),axis=1)
X_train=np.delete(X_train,4,axis=1)
X_test=np.concatenate((country_onehot_x_test,X_test),axis=1)
X_test=np.delete(X_test,4,axis=1)


In [11]:
X_train

array([[0.0, 1.0, 0.0, ..., 0.0, 1.0, 124749.08],
       [1.0, 0.0, 0.0, ..., 0.0, 0.0, 41104.82],
       [0.0, 1.0, 0.0, ..., 1.0, 1.0, 45750.21],
       ...,
       [1.0, 0.0, 0.0, ..., 1.0, 1.0, 92027.69],
       [1.0, 0.0, 0.0, ..., 1.0, 1.0, 101168.9],
       [0.0, 1.0, 0.0, ..., 1.0, 0.0, 33462.94]], dtype=object)

In [12]:
X_sc=StandardScaler()
X_sc=X_sc.fit(X_train)
X_train=X_sc.transform(X_train)
X_test=X_sc.transform(X_test)

In [13]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense

classifier= keras.models.Sequential()
classifier.add(Dense(units=7,activation='relu',input_shape=(12,)))
classifier.add(Dense(units=7,activation='relu'))
classifier.add(Dense(units=1,activation='sigmoid'))

classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [14]:
classifier.fit(X_train,Y_train,batch_size=64,epochs=50,verbose=1)

Epoch 1/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5819 - accuracy: 0.7479
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.4880 - accuracy: 0.7972
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.4613 - accuracy: 0.7972
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.4475 - accuracy: 0.7972
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.4386 - accuracy: 0.7972
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.4328 - accuracy: 0.7972
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.4282 - accuracy: 0.7972
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.4253 - accuracy: 0.7972
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.4233 - accuracy: 0.8136
Epoch 10/50
125/125 [==============================] - 0s 1ms/step - loss: 0.4215 - accuracy: 0.8181

In [15]:
classifier.evaluate(X_test,Y_test,batch_size=16)

125/125 [==============================] - 0s 912us/step - loss: 0.3384 - accuracy: 0.8640


[0.33840855956077576, 0.8640000224113464]

**Prediction**<br>
Whether a person having,<br>
Geography: France<br>
Credit Score: 600<br>
Gender: Male<br> 
Age: 40 years old<br>
Tenure: 3 years<br>
Balance: \$60000 <br>
Number of Products: 2<br>
Does he have a credit card ? Yes<br>
Is this customer an Active Member: Yes<br>
Estimated Salary: \$50000<br>

Will get the Loan?

In [16]:
new_data=np.array([1,0,0,600,1,40,3,6000,2,1,1,50000])
new_data

array([    1,     0,     0,   600,     1,    40,     3,  6000,     2,
           1,     1, 50000])

In [17]:
N_data=X_sc.transform(new_data.reshape(1,-1))


In [18]:
prediction=classifier.predict(N_data)
prediction=(prediction>0.5)
print(prediction)

[[False]]
